2023年度前期

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/banner.png?raw=true" height="120px">


# 解析実習 後半

　ここからは、ゲノム配列から任意の遺伝子配列を抜き出して、その遺伝子系統樹を作成し、種間・種内の系統関係を調べていきます。


## 解析実習の流れ
　このテキストでは「解析実習2」をおこないます。

- 解析実習1（シーケンシングリード処理、変異検出、配列決定） ※[前半パートのテキスト](https://colab.research.google.com/github/CropEvol/lecture/blob/master/FFBCexpr_2023spr/FFBC2023spr_CropEvol_p1.ipynb)
- 解析実習2（系統解析）
  1. 遺伝子配列をジーンバンクから探す（NCBI ワード検索）
  1. 遺伝子の相同配列をジーンバンクから探す（NCBI BLAST検索）
  1. 遺伝子の相同配列を実習サンプルのシーケンスから探す（ローカルBLAST検索）
  1. 集めた相同配列で系統樹を作る





## 事前準備: ソフトウェアをインストールする

　実習前に次のコードセルを実行してください。

In [ ]:
### 実習前にこのコードセルを実行してください
!wget -q -O Install_softwares_p2.sh https://raw.githubusercontent.com/CropEvol/lecture/master/FFBCexpr_2023spr/scripts/Install_softwares_p2.sh
!sh Install_softwares_p2.sh

## 事前準備: 実習サンプルのシーケンスのダウンロード

　次のコードセルを実行して、実習サンプルのシーケンスをダウンロードしてください。

In [ ]:
## Aegilops bicornis
!wget -q -O BIC_0.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/BIC_0.fasta?raw=true
!wget -q -O BIC_1.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/BIC_1.fasta?raw=true
!wget -q -O BIC_2.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/BIC_2.fasta?raw=true
## Aegilops longissima
!wget -q -O LON_0.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/LON_0.fasta?raw=true
!wget -q -O LON_2.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/LON_2.fasta?raw=true
## Aegilops sharonensis
!wget -q -O SHA_0.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/SHA_0.fasta?raw=true
## Aegilops searsii
!wget -q -O SEA_0.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/SEA_0.fasta?raw=true
## Dioscorea tokoro 東北サンプル
!wget -q -O NM.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/NM.fasta?raw=true
!wget -q -O NF.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/NF.fasta?raw=true
## Dioscorea tokoro 近畿サンプル
!wget -q -O CM.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/CM.fasta?raw=true
!wget -q -O CF.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/CF.fasta?raw=true
## Dioscorea tokoro 九州サンプル
!wget -q -O SM.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/SM.fasta?raw=true
!wget -q -O SF.fasta https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/dataset/fasta/SF.fasta?raw=true
!ls *.fasta

## 1. 遺伝子配列をジーンバンクから探す（NCBI ワード検索）

　1970年代にサンガー法が開発されて以降、世界に数箇所あるDNAデータバンクに塩基配列やアミノ酸配列等のデータが蓄積され続けています。また、それらデータは研究者が広く利用できるように公開されています。

おもなDNAデータバンク: 
- NCBI（アメリカ）: https://www.ncbi.nlm.nih.gov/
- ENA（イギリス）: https://www.ebi.ac.uk/ena/browser/
- DDBJ（日本）: https://www.ddbj.nig.ac.jp/index.html

　まず最初に、NCBIと呼ばれるアメリカのDNAデータバンクから、次の遺伝子の塩基配列データを探してみましょう。

- *rbcL*
  - ribulose-1,5-bisphosphate carboxylase/oxgenase large subunit
  - 葉緑体ゲノム上に存在する遺伝子で、光合成のための重要なタンパク質（Rubiscoの大サブユニット）をコードしている

　次のワードをNCBIのWebページで検索してください。
- コムギ近縁野生種 `rbcL Aegilops`
- オニドコロ `rbcL Dioscorea`

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/ncbi_word_search.png?raw=true" height="220px">


## 2. 遺伝子の相同配列をジーンバンクから探す（NCBI BLAST検索）

　次に、*rbcL* の相同配列を探す方法を学びましょう。Basic Local Alignment Search Tool (BLAST) というツールを利用することで、データベースにある相同な塩基配列を探すことができます。BLASTを使った相同性検索は、 **BLAST検索** と呼ばれることが多いです。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/blast_search.png?raw=true" height="220px">

　ここでは、rbcLの相同配列をNCBIのデータベースから探してみましょう。
- NCBI BLAST検索ページ: https://blast.ncbi.nlm.nih.gov/Blast.cgi

<small>※ BLAST検索にはいくつか種類があります。今回は、「Nucleotide BLAST」（塩基配列（クエリ） 対 塩基配列（データベース） の検索）を使います</small>

　NCBI BLAST検索の配列入力フォームに先ほど調べたrbcLの塩基配列をコピー＆ペーストし、下のほうにある「BLAST」ボタンを押してください。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/ncbi_blast_search.png?raw=true" height="350px">

## 3. 遺伝子の相同配列を実習サンプルのシーケンスから探す（ローカルBLAST検索）

　今度は、手元の実習サンプルのゲノム配列中のどこに *rbcL* の相同配列があるか探して、その *rbcL* 相同配列を取り出してみましょう。

### 3a. ローカルBLAST検索

　先ほどはNCBIのWebサービスによるBLAST検索でしたが、こちらは手元の配列データをデータベースにする ローカルBLAST検索と呼ばれる方法です。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/local_blast_search.png?raw=true" height="130px">

　NCBIから配布されているBLAST+というツール中の `blastn` でローカルBLAST検索を実施できます。

```sh
## blastnの使い方
blastn -subject SUBJECT.fasta -query QUERY.fasta
```

In [ ]:
%%bash
blastn 

### 3b. 相同配列を切り出す

　実習サンプルのゲノム配列中の *rbcL* の位置情報が得られたので、その情報を使って相同配列を取り出しましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/extract_subseq.png?raw=true" height="200px">

　ゲノム配列から任意の位置の配列を切り出すには、前半のステップ4でも使用した `samtools faidx` を使います。

```sh
## samtools faidxでの配列抽出
samtools faidx SUBJECT.fasta CHR:START-END > SUBSEQ.fasta

## 説明
- CHR: SUBJECT.fastaの塩基配列名
- START: 切り出す塩基範囲の始点
- END: 切り出す塩基範囲の終点
```

各実習サンプルの塩基配列名（`CHR`に入れる文字列）
- コムギ近縁野生種:
  - *Aegilops bicornis*: `BIC_0`, `BIC_1`, `BIC_2`
  - *Aegilops longissima*: `LON_0`, `LON_2`
  - *Aegilops sharonensis*: `SHA_0`
  - *Aegilops searsii*: `SEA_0`
- オニドコロ *Dioscorea tokoro*: 
  - 東北 雄株: `NM`
  - 東北 雌株: `NF`
  - 東北

In [ ]:
%%bash
samtools

　余裕がある方は、他のサンプルの *rbcL* 相同配列も集めてみてください。

## 4. 集めた相同配列で系統樹を作る
　集めた *rbcL* 相同配列を使って遺伝子系統樹を作成していきます。

- [コムギ近縁野生種の *rbcL* 相同配列セット](https://raw.githubusercontent.com/CropEvol/lecture/master/FFBCexpr_2023spr/dataset/Aegilops_rbcL_genes.fasta)
- [オニドコロの *rbcL* 相同配列セット](https://raw.githubusercontent.com/CropEvol/lecture/master/FFBCexpr_2023spr/dataset/Dioscorea_rbcL_genes.fasta)

<img src="https://github.com/CropEvol/lecture/blob/master/FFBCexpr_2023spr/images/gene_tree.png?raw=true" height="260px">

　今回の系統解析にはWebサービスを利用しましょう。入力フォームに相同配列情報を貼り付けて実行すると、簡単に系統解析をおこなえます。

- Phylogeny.fr: http://www.phylogeny.fr/simple_phylogeny.cgi
- Phylogenetic analysis pipeline by ETE3: https://www.genome.jp/tools-bin/ete

### 系統解析の詳細



　上述のWebサービスでは配列データを与えると、自動的に系統樹が作成されました。その自動処理では次のような処理がおこなわれています。

系統解析の工程: 
1. 塩基配列を相同部位で並べる（**マルチプルアライメント**）
1. 2つの塩基配列間で、異なる塩基の個数を調べて、塩基配列間の距離を求める
1. 塩基配列間の距離をもとに、系統樹を構築する

　次のWebツールを使うことで、今回の実習サンプルのマルチプルアライメントをおこない、塩基配列間の相違を確認できます。

- Clustal Omega: https://www.ebi.ac.uk/Tools/msa/clustalo/

## 後半まとめ＆演習
　解析実習の後半パートでは、系統解析の方法を学びました。
- NCBIワード検索で遺伝子配列を探す
- NCBI BLAST検索でDNAデータベース上の相同配列を探す
- ローカルBLAST検索で実習サンプルのシーケンスから相同配列を探す
- 遺伝子系統樹を作る

　他の遺伝子や遺伝子間領域でも実習サンプルの系統関係を調べてください。